In [2]:
# ! pip install webdriver-manager
# ! pip install fake-headers

In [15]:
from bs4 import BeautifulSoup
import random
import requests as req
import sys
import pandas as pd

# from fake_headers import Headers
# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
# from webdriver_manager.chrome import ChromeDriverManager
# from selenium.webdriver.chrome.options import Options

# s=Service(ChromeDriverManager().install())
# header = Headers(
#     browser="chrome",  # Generate only Chrome UA
#     os="win",  # Generate only Windows platform
#     headers=False # generate misc headers
# )
# customUserAgent = header.generate()['User-Agent']

# import webbrowser
# import time

import warnings
warnings.filterwarnings('ignore')

In [10]:
'''
Function which takes an url, parse it, assign a random user agent and then make a soup.
'''
def url_soup(u):
    try:
        uas = ("Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) Gecko/20100101 Firefox/40.1",
               "Mozilla/5.0 (Windows NT 6.3; rv:36.0) Gecko/20100101 Firefox/36.0",
               "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10; rv:33.0) Gecko/20100101 Firefox/33.0",
               "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36",
               "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.1 Safari/537.36",
               "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.0 Safari/537.36")
        ua = uas[random.randrange(len(uas))]
        headers = {'user-agent': ua}
        resp = req.get(u, headers=headers)
        c = resp.content
        soup = BeautifulSoup(c, features="html.parser")
        return soup
    except req.exceptions.HTTPError as e:
        print(e)
        sys.exit(1)

In [22]:
url = 'https://www.elmostrador.cl/noticias/pais/'
soup = url_soup(url)
for i in soup.find_all('a'):
    try:
        print(i['href'])
    except:
        pass

https://www.elmostrador.cl
#
https://www.elmostrador.cl/my-account/lost-password/
https://www.elmostrador.cl/laminutadelpoder/
https://www.elmostrador.cl/?s
https://www.elmostrador.cl
https://www.elmostrador.cl/mercados/
https://www.elmostrador.cl/tv/
https://www.elmostrador.cl/cultura/
https://www.elmostrador.cl/generacion-m/
https://www.elmostrador.cl/agenda-pais/
https://www.elmostrador.cl/braga/
http://legales.elmostrador.cl/
/noticias/pais/
#
https://www.facebook.com/elmostrador
https://twitter.com/elmostrador
https://instagram.com/el_mostrador
mailto:info@elmostrador.cl
https://www.elmostrador.cl/destacado/feed
https://www.elmostrador.cl
https://www.elmostrador.cl
https://www.elmostrador.cl/noticias/pais/
https://www.elmostrador.cl/noticias/mundo/
https://www.elmostrador.cl/noticias/opinion/
https://www.elmostrador.cl/noticias/sin-editar/
https://www.elmostrador.cl/noticias/deportes/
https://www.elmostrador.cl/noticias/multimedia/
https://www.elmostrador.cl/el-semanal/
/claves/pr

In [13]:
def get_body_text(list_of_keywords):
    chrome_options = Options()
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument("--disable-notifications")
    chrome_options.add_argument("--window-size=1920,1080")
    chrome_options.add_argument('--headless')
    chrome_options.add_argument(f'user-agent={customUserAgent}')
    browser = webdriver.Chrome('/Volumes/second/anatiri/clients/lmk/dev/chromedriver', options=chrome_options)
    # browser.set_window_size(1111, 1000)

    # Signing in
    browser.get('https://www.df.cl/')
    time.sleep(15)

    browser.find_element_by_xpath('/html/body/div[1]/div[4]/div[3]/div[1]/div[3]/div[1]/a').click()
    user_ = '/html/body/div[1]/div[4]/div[3]/div[1]/div[3]/div[2]/div[1]/form/input[4]'
    pass_ = '/html/body/div[1]/div[4]/div[3]/div[1]/div[3]/div[2]/div[1]/form/input[5]'
    time.sleep(3)
    browser.find_element_by_xpath(user_).send_keys('cris@crishernandez.co')
    browser.find_element_by_xpath(pass_).send_keys('@Kikaza1980PASS')
    browser.find_element_by_xpath('/html/body/div[1]/div[4]/div[3]/div[1]/div[3]/div[2]/div[1]/form/div/input[2]').click()
    time.sleep(3)

    # Get Article URLs
    keys = '+'.join(list_of_keywords)
    urls = []
    for _ in range(1,5):
        u = f'https://www.df.cl/cgi-bin/prontus_search.cgi?search_prontus=noticias&search_idx=all&search_tmp=search.html&search_form=yes&search_pag={_}&search_resxpag=30&search_maxpags=82&search_orden=cro&search_meta1=&search_meta2=&search_meta3=&search_seccion=&search_tema=&search_subtema=&search_fechaini=&search_fechafin=&search_texto={keys}&search_modo=and&search_comodines=no&vista='
        urls.append(u)

    appears = []
    for j in urls:
        try:
            browser.get(j)
            time.sleep(3)
            fs = BeautifulSoup(browser.page_source)
            articles = [f"https://www.df.cl{i.find_all('a')[-1]['href']}" for i in fs.find_all('article', {'class': 'noticia-horizontal'})]
            for article in articles:
                browser.get(article)
                time.sleep(3)
                ns = BeautifulSoup(browser.page_source)
                link = article
                title = ns.find_all('h1', {'class': 'titular'})[0].text.strip().lower()
                timestamp = ns.find_all('span', {'class': 'fecha-noticia'})[0].text.split('|')[-1].replace('Publicado:', '').strip()
                description = ns.find_all('p', {'class': 'bajada'})[0].text.replace(',',' ').lower().strip()

                o = {
                    'keywords': ' '.join(list_of_keywords),
                    'link': link,
                    'title': title,
                    'timestamp': timestamp,
                    'description': description
                }
                print(list_of_keywords)
                appears.append(o)
                time.sleep(3)
        except Exception as e:
            print(e)
            browser.close()
            pass

        browser.close()


    data = pd.DataFrame(appears, columns=['keywords','link','title','timestamp','description'])
    data.to_csv(f'../data/analysis/{"_".join(list_of_keywords)}.csv', index=False, sep=';')
    return appears

    # # Visiting each url and getting body text
    # browser.get(url)
    # time.sleep(3)
    # soup = BeautifulSoup(browser.page_source, "html.parser")
    # body = [i.find_all('p') for i in soup.find_all('div', {'class': 'CUERPO'})]
    # p = [i for i in body]
    # t = [i.text.strip().lower() for i in p[0]]
    # r = ' '.join(t)
    # browser.close()
    # return r

In [14]:
'''
List of chilean MF
'''
# path_to_mf_file = '../data/analysis/MULTIFAMILY.xlsx'
# mfs = pd.read_excel(path_to_mf_file)
# names_of_multi_families = [i.lower() + ' multifamily' for i in mfs.Multifamily.values]
# print(names_of_multi_families)

'''
Getting articles related to each MF
'''
to_search = 'multifamily'
to_search_formatted = to_search.split(' ')
# news = get_news_from_df(to_search_formatted, saving=True)
news = get_body_text(to_search_formatted)
news

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[1]/div[4]/div[3]/div[1]/div[3]/div[1]/a"}
  (Session info: headless chrome=97.0.4692.99)


In [153]:
'''
Open urls to test
'''
webbrowser.open(news.link.values[10])

True

In [155]:
'''
Get text body to identify names and numbers
'''
df = news.copy()
df

,keywords,link,title,timestamp,description
0,multifamily,https://www.df.cl/noticias/mercados/banca-fint...,se activa la grúa veraniega: altafid levanta a...,Martes 18 de enero de 2022 a las 13:39 hrs.,además abaqus contrató a exdirector de prochil...
1,multifamily,https://www.df.cl/noticias/empresas/industria/...,desde la uf congelada a flexibilidades en el p...,Martes 18 de enero de 2022 a las 12:41 hrs.,de acuerdo a algunos actores del mercado la i...
2,multifamily,https://www.df.cl/noticias/mercados/bolsa-mone...,compass arremete en renta residencial de bogot...,Lunes 17 de enero de 2022 a las 04:00 hrs.,ccla es la plataforma de inversión inmobiliari...
3,multifamily,https://www.df.cl/noticias/empresas/construcci...,farellones planea crear polo inmobiliario de m...,Lunes 17 de enero de 2022 a las 04:00 hrs.,mientras prepara las obras de la segunda fase ...
4,multifamily,https://www.df.cl/noticias/mercados/bolsa-mone...,los multifamily office chilenos que ganan terreno,Lunes 10 de enero de 2022 a las 04:00 hrs.,el mapa de las 12 de las oficinas que más terr...
...,...,...,...,...,...
56,multifamily,https://www.df.cl/noticias/mercados/banca-fint...,bci no descarta nuevas adquisiciones para crec...,Viernes 7 de febrero de 2020 a las 13:51 hrs.,la compañía afirmó que están priorizando su fo...
57,multifamily,https://www.df.cl/noticias/mercados/bolsa-mone...,conflicto social potencia creación de fondos c...,Martes 28 de enero de 2020 a las 04:00 hrs.,además de la invención de más de una decena de...
58,multifamily,https://www.df.cl/noticias/empresas/construcci...,fg revela detalles de proyecto con territoria ...,Jueves 14 de marzo de 2019 a las 04:00 hrs.,empresas evalúan segunda iniciativa en terreno...
59,multifamily,https://www.df.cl/noticias/empresas/actualidad...,el grupo valcan lanza tres nuevas marcas para ...,Miércoles 13 de febrero de 2019 a las 04:00 hrs.,


In [259]:
x = get_body_text(news.link.values[10])
print(x)

a casi dos años desde el primer caso de covid-19, y pese a la recuperación económica durante 2021, el sector hotelero aún no logra reponerse y sigue sin recuperar los ingresos que percibía antes de la pandemia. así lo revela la última encuesta elaborada por hoteleros de chile, gremio que encuestó a 48 establecimientos a lo largo del territorio nacional respecto de las reservas para la temporada de verano (diciembre 2021 a febrero 2022). de acuerdo al reporte, la tarifa de temporada a nivel nacional es, en promedio, un 20% más baja que en 2019. y en términos de ocupación, localidades como san pedro de atacama (30%) y la patagonia (31%) anotan las reservas más bajas del período de verano. por el contrario, entre las regiones con la mayor cantidad de reservas figuran las de los lagos (50%) y del biobío (52%).  el gremio proyecta que la situación más crítica tendrá lugar en enero próximo, donde se prevén ocupaciones de apenas un 20% en san pedro de atacama y en el extremo austral, en tanto